# Exercise 2

Tianfang Wang a1788845

## Loading modules

In [1]:
import os
import re
# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
from git import RemoteProgress
from git import Repo


## Case 1 CVE-2015-5174

In [2]:
repo1 = Repo("~/Documents/tomcat80")
fixing_commit_1 = "2fc9d03ffbc3fe7eabfd272380807ac0ddcf748d"

## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [3]:
res1 = repo1.git.log(-1, '--pretty=%B', fixing_commit_1)

print("Title:",res1)

Title: Fix Javadoc
Separate test cases into separate test methods and add some additional tests.
Clean up the code and fix a couple of edge cases

git-svn-id: https://svn.apache.org/repos/asf/tomcat/tc8.0.x/trunk@1696281 13f79535-47bb-0310-9956-ffa450edef68



## Q(b) How many total files were affected in the fixing commit? 

In [4]:
res2 = repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines()
print("Number of affected files: %d"%len(res2))

Number of affected files: 3


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [5]:
res3 = repo1.git.diff('--dirstat', fixing_commit_1, fixing_commit_1 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 3


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [6]:
res4 = repo1.git.show(fixing_commit_1).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 21


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [7]:
res5 = repo1.git.show(fixing_commit_1).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 98


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [8]:
res6 = repo1.git.show('--ignore-blank-lines', fixing_commit_1).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 20


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [9]:
res7 = repo1.git.show('--ignore-blank-lines', fixing_commit_1).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 78


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [10]:
for file in repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines():
    res8 = repo1.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 For java/org/apache/tomcat/util/http/RequestUtil.java , the days between the current fixing commit and the previous commit is 230.7 days
 For test/org/apache/tomcat/util/http/TestRequestUtil.java , the days between the current fixing commit and the previous commit is 571.9 days
 For webapps/docs/changelog.xml , the days between the current fixing commit and the previous commit is 26.1 days


## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [11]:
for file in repo1.git.diff('--name-only', fixing_commit_1, fixing_commit_1 + '^').splitlines():
    res9 = repo1.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  java/org/apache/tomcat/util/http/RequestUtil.java has been modified 6 times
  test/org/apache/tomcat/util/http/TestRequestUtil.java has been modified 14 times
  webapps/docs/changelog.xml has been modified 2919 times


## Q(j) Which developers have modified each affected file since its creation?

In [12]:
for file in repo1.git.diff('--name-only',fixing_commit_1, fixing_commit_1 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo1.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  java/org/apache/tomcat/util/http/RequestUtil.java has been modified by following developers:
    Mark Thomas
  test/org/apache/tomcat/util/http/TestRequestUtil.java has been modified by following developers:
    Mark Thomas
    Konstantin Kolinko
    Mladen Turk
    Remy Maucherat
  webapps/docs/changelog.xml has been modified by following developers:
    Remy Maucherat
    Christopher Schultz
    kfujino
    Konstantin Preißer
    Filip Hanik
    Keiichi Fujino
    Rainer Jung
    William Barker
    Felix Schumacher
    Mark Thomas
    Jean-Frederic Clere
    Emmanuel Bourg
    Violeta Georgieva Georgieva
    Yoav Shapira
    Konstantin Kolinko
    Coty Sutherland
    Tim Whittington
    Peter Rossbach
    Jeremy Boynes
    Mladen Turk
    Timothy A. Funk
    Sylvain Laurent


## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [13]:
devs = []
for file in repo1.git.diff('--name-only',fixing_commit_1, fixing_commit_1 + '^').splitlines():
    devs += repo1.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo1.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 Remy Maucherat made 655 commits
 Christopher Schultz made 81 commits
 kfujino made 1 commits
 Konstantin Preißer made 71 commits
 Filip Hanik made 1097 commits
 Keiichi Fujino made 389 commits
 Rainer Jung made 448 commits
 William Barker made 44 commits
 Felix Schumacher made 104 commits
 Mark Thomas made 10527 commits
 Jean-Frederic Clere made 81 commits
 Emmanuel Bourg made 7 commits
 Violeta Georgieva Georgieva made 340 commits
 Yoav Shapira made 8 commits
 Konstantin Kolinko made 1579 commits
 Coty Sutherland made 9 commits
 Tim Whittington made 23 commits
 Peter Rossbach made 93 commits
 Jeremy Boynes made 72 commits
 Mladen Turk made 129 commits
 Timothy A. Funk made 61 commits
 Sylvain Laurent made 30 commits


## Case 2 CVE-2018-1325

In [14]:
repo2 = Repo("~/Documents/wicket-jquery-ui")
fixing_commit_2 = "2fec03dbe2f6e8808f4bdc6b3195dff3e44f520"

## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [15]:
res1 = repo2.git.log(-1, '--pretty=%B', fixing_commit_2)

print("Title:",res1)

Title: Plugins: WysiwygEditor, replaced textarea by hidden input



## Q(b) How many total files were affected in the fixing commit? 

In [16]:
res2 = repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines()
print("Number of affected files: %d"%len(res2))

Number of affected files: 2


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [17]:
res3 = repo2.git.diff('--dirstat', fixing_commit_2, fixing_commit_2 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 1


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [18]:
res4 = repo2.git.show(fixing_commit_2).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 12


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [19]:
res5 = repo2.git.show(fixing_commit_2).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 6


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [20]:
res6 = repo2.git.show('--ignore-blank-lines', fixing_commit_2).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 12


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [21]:
res7 = repo2.git.show('--ignore-blank-lines', fixing_commit_2).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 6


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [22]:
for file in repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines():
    res8 = repo2.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 For wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html , the days between the current fixing commit and the previous commit is 1702.0 days
 For wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java , the days between the current fixing commit and the previous commit is 47.7 days


## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [23]:
for file in repo2.git.diff('--name-only', fixing_commit_2, fixing_commit_2 + '^').splitlines():
    res9 = repo2.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html has been modified 6 times
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java has been modified 26 times


## Q(j) Which developers have modified each affected file since its creation?

In [24]:
for file in repo2.git.diff('--name-only',fixing_commit_2, fixing_commit_2 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo2.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.html has been modified by following developers:
    sbriquet
    Sebastien Briquet
    Sebastien
  wicket-jquery-ui-plugins/src/main/java/com/googlecode/wicket/jquery/ui/plugins/wysiwyg/WysiwygEditor.java has been modified by following developers:
    sbriquet
    Sebastien
    Maxim Solodovnik
    Sebastien Briquet


## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [25]:
devs = []
for file in repo2.git.diff('--name-only',fixing_commit_2, fixing_commit_2 + '^').splitlines():
    devs += repo2.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo2.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 sbriquet made 162 commits
 Sebastien Briquet made 489 commits
 Maxim Solodovnik made 24 commits
 Sebastien made 963 commits


## Case 3 CVE-2013-7251

In [26]:
repo3 = Repo("~/Documents/projectforge-webapp")
fixing_commit_3 = "422de35e3c3141e418a73bfb39b430d5fd74077e"

## Q(a) What was the message and title of the fixing commit? Was there any mention of fixing a bug or vulnerability? 

In [27]:
res1 = repo3.git.log(-1, '--pretty=%B', fixing_commit_3)

print("Title:",res1)

Title: CSRF protection.



## Q(b) How many total files were affected in the fixing commit? 

In [28]:
res3 = repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines()
print("Number of affected files: %d"%len(res3))

Number of affected files: 20


## Q(c) How many total directories were affected in the fixing commit? For example, if a file path is: abc/def/File.java, then its directory is abc/def.

In [29]:
res3 = repo3.git.diff('--dirstat', fixing_commit_3, fixing_commit_3 + '^').splitlines()
print("Number of affected directories: %d"%len(res3))

Number of affected directories: 8


## Q(d) How many total lines of code (including comments and blank lines) were deleted? 

In [30]:
res4 = repo3.git.show(fixing_commit_3).splitlines()
res4 = [line for line in res4 if re.compile('^-$|^-[^-]').match(line)]
print("Number of deleted lines (including comments and blank lines): %d"%len(res4))

Number of deleted lines (including comments and blank lines): 20


## Q(e) How many total lines of code (including comments and blank lines) were added?

In [31]:
res5 = repo3.git.show(fixing_commit_3).splitlines()
res5 = [line for line in res5 if re.compile('^\+$|^\+[^\+]').match(line)]
print("Number of added lines (including comments and blank lines): %d"%len(res5))

Number of added lines (including comments and blank lines): 165


## Q(f) How many total lines of code (excluding comments and blank lines) were deleted? 

In [32]:
res6 = repo3.git.show('--ignore-blank-lines', fixing_commit_3).splitlines()
res6 = [line for line in res6 if re.compile('^-[^-]').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*/').match(line)]
res6 = [line for line in res6 if not re.compile('^[-|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res6))

Number of added lines (excluding comments and blank lines): 20


## Q(g) How many total lines of code (excluding comments and blank lines) were added? 

In [33]:
res7 = repo3.git.show('--ignore-blank-lines', fixing_commit_3).splitlines()
res7 = [line for line in res7 if re.compile('^\+[^\+]').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*/').match(line)]
res7 = [line for line in res7 if not re.compile('^[\+|]\s*\*').match(line)]
print("Number of added lines (excluding comments and blank lines): %d"%len(res7))

Number of added lines (excluding comments and blank lines): 119


## Q(h) How many days were between the current fixing commit and the previous commit of each affected file?

In [34]:
for file in repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines():
    res8 = repo3.git.log(-2, '--pretty=%ct', file).splitlines();
    if len(res8) == 2:
        print(" For %s , the days between the current fixing commit and the previous commit is %.1f days"%(file, (int(res8[0])-int(res8[1]))/86400))
    else:
        print(" %s:  N/A This file doesn't have previous commit"%file)

 For src/main/java/org/projectforge/web/admin/SetupForm.java , the days between the current fixing commit and the previous commit is 39.0 days
 For src/main/java/org/projectforge/web/admin/SetupImportForm.java , the days between the current fixing commit and the previous commit is 78.7 days
 For src/main/java/org/projectforge/web/admin/SetupPage.html , the days between the current fixing commit and the previous commit is 304.3 days
 For src/main/java/org/projectforge/web/admin/SystemUpdateForm.java , the days between the current fixing commit and the previous commit is 78.7 days
 For src/main/java/org/projectforge/web/admin/SystemUpdatePage.html , the days between the current fixing commit and the previous commit is 304.3 days
 For src/main/java/org/projectforge/web/core/NavTopPanel.html , the days between the current fixing commit and the previous commit is 28.5 days
 For src/main/java/org/projectforge/web/core/NavTopPanel.java , the days between the current fixing commit and the prev

## Q(i) How many time has each affected file of the current fixing commit been modified in the past since their creation  

In [35]:
for file in repo3.git.diff('--name-only', fixing_commit_3, fixing_commit_3 + '^').splitlines():
    res9 = repo3.git.log('--follow', '--pretty=oneline', file).splitlines();
    print("  %s has been modified %d times"%(file, len(res9)))

  src/main/java/org/projectforge/web/admin/SetupForm.java has been modified 25 times
  src/main/java/org/projectforge/web/admin/SetupImportForm.java has been modified 9 times
  src/main/java/org/projectforge/web/admin/SetupPage.html has been modified 16 times
  src/main/java/org/projectforge/web/admin/SystemUpdateForm.java has been modified 25 times
  src/main/java/org/projectforge/web/admin/SystemUpdatePage.html has been modified 11 times
  src/main/java/org/projectforge/web/core/NavTopPanel.html has been modified 39 times
  src/main/java/org/projectforge/web/core/NavTopPanel.java has been modified 38 times
  src/main/java/org/projectforge/web/dialog/ModalDialog.html has been modified 16 times
  src/main/java/org/projectforge/web/dialog/ModalDialog.java has been modified 33 times
  src/main/java/org/projectforge/web/fibu/RechnungCostEditTablePanel.html has been modified 6 times
  src/main/java/org/projectforge/web/fibu/RechnungCostEditTablePanel.java has been modified 8 times
  src/ma

## Q(j) Which developers have modified each affected file since its creation?

In [36]:
for file in repo3.git.diff('--name-only',fixing_commit_3, fixing_commit_3 + '^').splitlines():
    print("  %s has been modified by following developers:"%file)
    res10 = repo3.git.log('--follow', '--pretty=%aN', file).splitlines()
    for dev in set(res10):
        print("    %s"%dev)

  src/main/java/org/projectforge/web/admin/SetupForm.java has been modified by following developers:
    Kai Reinhard
    kai
  src/main/java/org/projectforge/web/admin/SetupImportForm.java has been modified by following developers:
    Kai Reinhard
    kai
  src/main/java/org/projectforge/web/admin/SetupPage.html has been modified by following developers:
    Kai Reinhard
    Dennis Hilpmann
    kai
  src/main/java/org/projectforge/web/admin/SystemUpdateForm.java has been modified by following developers:
    Kai Reinhard
    kai
  src/main/java/org/projectforge/web/admin/SystemUpdatePage.html has been modified by following developers:
    Kai Reinhard
    kai
  src/main/java/org/projectforge/web/core/NavTopPanel.html has been modified by following developers:
    Dennis Hilpmann
    dhilpmann
    Kai Reinhard
    kai
  src/main/java/org/projectforge/web/core/NavTopPanel.java has been modified by following developers:
    Johannes Unterstein
    Kai Reinhard
    kai
    junterstein
  

## Q(k) For each developer identified, how many commits have each of them submitted? From your observation, are the involving developers experienced (with many commits) or new ones (with few commits) or both? 

In [37]:
devs = []
for file in repo3.git.diff('--name-only',fixing_commit_3, fixing_commit_3 + '^').splitlines():
    devs += repo3.git.log('--follow', '--pretty=%aN', file).splitlines()
res11 = list(set(devs))
log = repo3.git.log('--pretty=%aN')
for dev in res11:
    print(" %s made %d commits"%(dev, len(re.findall(dev,log))))

 markus made 41 commits
 Johannes Unterstein made 155 commits
 dhilpmann made 16 commits
 Kai Reinhard made 1420 commits
 kai made 4074 commits
 junterstein made 196 commits
 Dennis Hilpmann made 40 commits
